# Fine-tuning BERT on TSV Files
This notebook demonstrates how to fine-tune a BERT model using all TSV files from the `./learning/ready/` directory.

In [1]:
# Import necessary libraries
import os
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

In [5]:
data_dir = './ready/'
all_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.tsv')]
print(pd.read_csv(all_files[0], sep='\t').columns)

Index(['label', 'text'], dtype='object')


In [ ]:
# Load and preprocess data
data_dir = './ready/'
all_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.tsv')]

dataframes = []
for file in all_files:
    df = pd.read_csv(file, sep='\t')
    dataframes.append(df)

# Combine all dataframes
data = pd.concat(dataframes, ignore_index=True)

# Assuming the TSV files have 'text' and 'label' columns
labels = data['label'].tolist()
texts = data['text'].tolist()

# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [ ]:
# Tokenize the data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

In [ ]:
# Convert data to PyTorch Dataset
import torch

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)

In [ ]:
# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(set(labels)))

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Save the model
model.save_pretrained('./learning/trained_models/fine_tuned_bert')
tokenizer.save_pretrained('./learning/trained_models/fine_tuned_bert')